In [21]:
import os
import json

from pathlib import Path
import pandas as pd

from quixbugs import QuixBugsDataset
from utils import cleaning_code

exp_names = [
    "gpt35-python-basic",
    "gpt35-python-with_lib_v2",
    "gpt35-python-with_location",
    "gpt4-python-basic_v2",
    "gpt4-python-with_lib_v3",
    "gpt4-python-with_location",
    "gemini-python-basic",
    "gemini-python-with_lib_v2",
    "gemini-python-with_location",
    "claude-python-basic",
]

exp_name = exp_names[3]

base_dir = Path(str(os.getenv("EXPS_DIR")))
eval_art_dir = base_dir / exp_name / "evals_art"
eval_dir = base_dir / exp_name / "evals"

if not eval_art_dir.is_dir():
    eval_art_dir.mkdir()


In [22]:
dataset_dir = Path(str(os.getenv("QUIX_BUGS_DIR")))
dataset = QuixBugsDataset('python')
with open(dataset_dir / "prog_info.json", "r") as file:
    prog_info: list[dict] = json.load(file)
prog_names = [i["prog_name"] for i in prog_info]

prog_names


['bitcount',
 'breadth_first_search',
 'bucketsort',
 'depth_first_search',
 'detect_cycle',
 'find_first_in_sorted',
 'find_in_sorted',
 'flatten',
 'gcd',
 'get_factors',
 'hanoi',
 'is_valid_parenthesization',
 'kheapsort',
 'knapsack',
 'kth',
 'lcs_length',
 'levenshtein',
 'lis',
 'longest_common_subsequence',
 'max_sublist_sum',
 'mergesort',
 'minimum_spanning_tree',
 'next_palindrome',
 'next_permutation',
 'pascal',
 'possible_change',
 'powerset',
 'quicksort',
 'reverse_linked_list',
 'rpn_eval',
 'shortest_path_length',
 'shortest_path_lengths',
 'shortest_paths',
 'shunting_yard',
 'sieve',
 'sqrt',
 'subsequences',
 'to_base',
 'topological_ordering',
 'wrap']

In [23]:
df = pd.DataFrame({i: "F" for i in ["patch_0", "patch_1", "patch_2"]}, index=prog_names)

eval_art_dicts = {}

for sample in dataset:
    eval_art_dict = {
        "sample": sample.to_dict(),
        "patchs_art_evals": {
            "patch-0": {
                "is_correct": "F",
                "bug_location_predict": 0,
                "patch_fail_detail": "",
                "patch_correct_detail": "",
            },
            "patch-1": {
                "is_correct": "F",
                "bug_location_predict": 0,
                "patch_fail_detail": "",
                "patch_correct_detail": "",
            },
            "patch-2": {
                "is_correct": "F",
                "bug_location_predict": 0,
                "patch_fail_detail": "",
                "patch_correct_detail": "",
            },
        },
    }

    eval_art_dicts[f"{sample.prog_id}. {sample.prog_name}.json"] = eval_art_dict

eval_art_dicts

{'0. bitcount.json': {'sample': {'prog_id': 0,
   'prog_name': 'bitcount',
   'language': 'python',
   'testcase_num': 9,
   'fault_location': 3,
   'bug_type': '',
   'bug_detail_desc': '在運算消除n中的1時，應該用and運算卻用成xor，造成原本應該能消掉1的步驟變成n只能保持不變，導致while迴圈無限循環'},
  'patchs_art_evals': {'patch-0': {'is_correct': 'F',
    'bug_location_predict': 0,
    'patch_fail_detail': '',
    'patch_correct_detail': ''},
   'patch-1': {'is_correct': 'F',
    'bug_location_predict': 0,
    'patch_fail_detail': '',
    'patch_correct_detail': ''},
   'patch-2': {'is_correct': 'F',
    'bug_location_predict': 0,
    'patch_fail_detail': '',
    'patch_correct_detail': ''}}},
 '1. breadth_first_search.json': {'sample': {'prog_id': 1,
   'prog_name': 'breadth_first_search',
   'language': 'python',
   'testcase_num': 5,
   'fault_location': 9,
   'bug_type': '',
   'bug_detail_desc': '在while迴圈的判斷式中，應該使用while queue讓佇列數量為0時脫離迴圈卻錯用為while True，導致迴圈無限循環無法停止'},
  'patchs_art_evals': {'patch-0': {'is_correct': 'F',
    '

In [24]:
all_code_flie = {}
plausible_check = {}

all_eval_file = eval_dir.glob("*")
for eval_number in all_eval_file:
    with open(eval_number) as f:
        eval_info = json.load(f)

    prog_name = eval_info["sample"]["prog_name"]
    prog_id = eval_info["sample"]["prog_id"]
    sample = dataset[prog_id]
    correct_code = cleaning_code(sample.correct_code)
    eval_art_dict = eval_art_dicts[f"{sample.prog_id}. {sample.prog_name}.json"]

    all_code_flie[f"{prog_id}. {prog_name}-patch-correct.py"] = correct_code

    patchs_eval = eval_info["patchs_eval"]
    for eval_number, patch_eval in enumerate(patchs_eval):
        reparied_code = patch_eval["reparied_code"]
        all_code_flie[f"{prog_id}. {prog_name}-patch-{eval_number}.py"] = reparied_code

        # TODO consider to change to pass_num with testcase_num
        is_plausible = patch_eval["is_plausible"]
        is_correct = patch_eval["is_correct"]

        patch_eval_art = eval_art_dict["patchs_art_evals"][f"patch-{eval_number}"]
        if is_plausible and is_correct:
            patch_eval_art["is_correct"] = "T"
        elif is_plausible:
            patch_eval_art["is_correct"] = "U"


# all_code_flie
eval_art_dicts

{'0. bitcount.json': {'sample': {'prog_id': 0,
   'prog_name': 'bitcount',
   'language': 'python',
   'testcase_num': 9,
   'fault_location': 3,
   'bug_type': '',
   'bug_detail_desc': '在運算消除n中的1時，應該用and運算卻用成xor，造成原本應該能消掉1的步驟變成n只能保持不變，導致while迴圈無限循環'},
  'patchs_art_evals': {'patch-0': {'is_correct': 'T',
    'bug_location_predict': 0,
    'patch_fail_detail': '',
    'patch_correct_detail': ''},
   'patch-1': {'is_correct': 'T',
    'bug_location_predict': 0,
    'patch_fail_detail': '',
    'patch_correct_detail': ''},
   'patch-2': {'is_correct': 'T',
    'bug_location_predict': 0,
    'patch_fail_detail': '',
    'patch_correct_detail': ''}}},
 '1. breadth_first_search.json': {'sample': {'prog_id': 1,
   'prog_name': 'breadth_first_search',
   'language': 'python',
   'testcase_num': 5,
   'fault_location': 9,
   'bug_type': '',
   'bug_detail_desc': '在while迴圈的判斷式中，應該使用while queue讓佇列數量為0時脫離迴圈卻錯用為while True，導致迴圈無限循環無法停止'},
  'patchs_art_evals': {'patch-0': {'is_correct': 'U',
    '

In [25]:
temp_dir = Path("../temp")

for file_name, code in all_code_flie.items():
    with open(temp_dir / file_name, "w") as f:
        f.write(code)


# df.to_csv(eval_art_dir / "is_correct.csv")
for file_name, json_content in eval_art_dicts.items():
    num, *_ = file_name.split('.')
    num = int(num)

    if num >= 9:
        with open(eval_art_dir / file_name, "w", encoding="UTF-8") as f:
            json.dump(json_content, f, indent=4, ensure_ascii=False)
